In [2]:
import os   
from pathlib import Path
from dotenv import load_dotenv
from rich.progress import track  # rich 진행바 임포트
from autoLeague.replays.scraper import ReplayScraper


load_dotenv()  
API_KEY = os.environ["API_KEY"]

# ── 공통 경로 설정(Set common base path) ─────────────────────────────────────
USER_HOME = Path(os.environ["USERPROFILE"])    # C:\Users\<계정명>

GAME_DIR      = Path(r"C:\Riot Games\League of Legends\Game")
REPLAY_DIR    = str(USER_HOME / "Documents" / "League of Legends" / "Replays")
SAVE_DIR      = 'recordings_game'
SCRAPER_DIR   = str(USER_HOME / "Desktop" / "pyLoL" / "pyLoL" / "autoLeague" / "replays")
MATCH_INFO_DIR = str(USER_HOME / "Desktop" / "pyLoL" / "data" / "match_info")

rs = ReplayScraper(game_dir=GAME_DIR, replay_dir=REPLAY_DIR, save_dir=SAVE_DIR, scraper_dir=SCRAPER_DIR, replay_speed=8, region="KR")


현재 있는 리플레이 갯수 :  6893


In [ ]:
import json
import os
from rich.console import Console
from rich.progress import Progress, BarColumn, TextColumn, TimeRemainingColumn

# 콘솔 설정 (너비 150)x
console = Console(width=150)

# JSON 파일에서 데이터를 읽음
json_path = os.path.join('data', 'filltered_kill_events_timeline_filtered_10up_1.json')
with open(json_path, 'r', encoding='utf-8') as f:
    data_dict = json.load(f)['data']

# 실행할 태스크 목록 생성: 각 태스크는 (replay, start, end)의 튜플
tasks = []


for match_id, minutes in data_dict.items():
    # replay 파일명은 "KR-<match_id>.rofl" 형태
        replay = f"KR-{match_id}.rofl"
        for minute in minutes:
            start = (minute+1) * 60 - 3  # 시간 보정을 위해 -5초
            end = start + 60 - 8        # end는 start + 60초
            tasks.append((replay, start, end))


# Rich progress bar를 사용하여 태스크 순차 실행
with Progress(
    TextColumn("[progress.description]{task.description}"),
    BarColumn(),
    TimeRemainingColumn(),
    console=console,
    transient=True,
) as progress:
    task_progress = progress.add_task("Extracting Replay_Minimaps from LoL Client...", total=len(tasks))
    for replay, start, end in tasks:
        rs.run_client_ver1(
            replay_path=rf'{REPLAY_DIR}\{replay}',
            gameId=replay.split('.')[0],
            start=start,
            end=end,
            speed=8,
            paused=False,
            team="All",
            use_nas=False,
            remove_fog_of_war=True
        )
        progress.advance(task_progress)

Output()